In [8]:
# ---------------------------
# SIMPLE FOIL EXAMPLE
# ---------------------------

# Background facts (known relations)
parents = [
    ('John', 'Mary'),
    ('Mary', 'Sam'),
    ('Sam', 'Alice')
]

# Positive examples of target relation (Ancestor)
positives = [
    ('John', 'Mary'),
    ('Mary', 'Sam'),
    ('Sam', 'Alice'),
    ('John', 'Sam'),
    ('Mary', 'Alice'),
    ('John', 'Alice')
]

# ---------------------------
# Step 1: Start with simplest rule
# ---------------------------

rules = []

# Rule 1: Direct relation
# If Parent(X,Y) then Ancestor(X,Y)
for (x, y) in parents:
    if ('Ancestor', x, y) not in rules:
        rules.append(('Ancestor', [('Parent', 'X', 'Y')]))

# Rule 2: Recursive relation
# If Parent(X,Z) and Ancestor(Z,Y) then Ancestor(X,Y)
rules.append(('Ancestor', [('Parent', 'X', 'Z'), ('Ancestor', 'Z', 'Y')]))

# ---------------------------
# Step 2: Print the rules
# ---------------------------

print("===== Learned FOIL Rules =====\n")
for i, (head, body) in enumerate(rules, start=1):
    body_str = " ∧ ".join(f"{r}({x},{y})" for (r, x, y) in body)
    print(f"Rule {i}: IF {body_str} THEN {head}(X,Y)")

print("\nLearning complete.")


===== Learned FOIL Rules =====

Rule 1: IF Parent(X,Y) THEN Ancestor(X,Y)
Rule 2: IF Parent(X,Y) THEN Ancestor(X,Y)
Rule 3: IF Parent(X,Y) THEN Ancestor(X,Y)
Rule 4: IF Parent(X,Z) ∧ Ancestor(Z,Y) THEN Ancestor(X,Y)

Learning complete.


In [9]:
# ============================
# RULE LEARNING DEMONSTRATION
# ============================

!pip install orange3 --quiet

import pandas as pd
from sklearn.datasets import load_iris
from Orange.data import Domain, Table, ContinuousVariable, DiscreteVariable
from Orange.classification.rules import CN2Learner



# ============================
# PART 2: FOIL (FIRST ORDER INDUCTIVE LEARNING)
# ============================

print("\n===== FOIL RULE LEARNING (First-Order Logic) =====\n")

# Example: family relations (background knowledge)
background = [
    ('Parent', 'John', 'Mary'),
    ('Parent', 'Mary', 'Sam'),
    ('Parent', 'Sam', 'Alice'),
]

# Positive examples (target relation)
positives = [
    ('Ancestor', 'John', 'Mary'),
    ('Ancestor', 'Mary', 'Sam'),
    ('Ancestor', 'Sam', 'Alice'),
    ('Ancestor', 'John', 'Sam'),
    ('Ancestor', 'Mary', 'Alice'),
    ('Ancestor', 'John', 'Alice'),
]

def foil_learn(target_relation, positives, background):
    """Simple demonstration of FOIL-like rule learning."""
    rules = []
    uncovered = set(positives)

    while uncovered:
        rule_body = []
        current_covered = set()

        # Try to find direct and recursive rules
        for rel, x, y in uncovered:
            if rel == target_relation:
                if ('Parent', x, y) in background:
                    # Direct rule
                    rule_body = [('Parent', 'X', 'Y')]
                    current_covered.add((rel, x, y))
                else:
                    # Recursive rule
                    for _, a, z in background:
                        for r2, z2, y2 in positives:
                            if r2 == target_relation and z == z2 and y == y2:
                                rule_body = [('Parent', 'X', 'Z'), ('Ancestor', 'Z', 'Y')]
                                current_covered.add((rel, a, y2))
        if rule_body:
            rules.append((target_relation, rule_body))
        uncovered -= current_covered

    return rules

# Learn FOIL-style rules
foil_rules = foil_learn('Ancestor', positives, background)

# Display FOIL rules
for i, (head, body) in enumerate(foil_rules, start=1):
    body_str = " ∧ ".join(f"{r}({x},{y})" for (r, x, y) in body)
    print(f"Rule {i}: IF {body_str} THEN {head}(X,Y)")

print("\nLearning complete.")



===== FOIL RULE LEARNING (First-Order Logic) =====

Rule 1: IF Parent(X,Y) THEN Ancestor(X,Y)

Learning complete.
